In [34]:
# webpage extraction with beautifulsoup
import urllib2
from bs4 import BeautifulSoup

#specify the url
manual = 'https://haynes.com/en-us/car-manuals/car-makes'

#Query the website and return the html to the variable 'page'
page = urllib2.urlopen(manual)

In [35]:
soup = BeautifulSoup(page)

In [ ]:
print soup.prettify()

In [75]:
makes = soup.find_all("div", class_="item-list")[1]
links_makes = makes.find_all("div", class_ = "views-field views-field-nothing")

In [80]:
list_makes = []
prefix = "https://haynes.com"
for make in links_makes:
    list_makes.append(prefix + make.a["href"])    

['https://haynes.com/en-us/car-manuals/acura', 'https://haynes.com/en-us/car-manuals/amc', 'https://haynes.com/en-us/car-manuals/audi', 'https://haynes.com/en-us/car-manuals/austin-healey', 'https://haynes.com/en-us/car-manuals/bmw', 'https://haynes.com/en-us/car-manuals/buick', 'https://haynes.com/en-us/car-manuals/cadillac', 'https://haynes.com/en-us/car-manuals/chevrolet', 'https://haynes.com/en-us/car-manuals/chrysler', 'https://haynes.com/en-us/car-manuals/datsun', 'https://haynes.com/en-us/car-manuals/dodge', 'https://haynes.com/en-us/car-manuals/eagle', 'https://haynes.com/en-us/car-manuals/fiat', 'https://haynes.com/en-us/car-manuals/ford', 'https://haynes.com/en-us/car-manuals/geo', 'https://haynes.com/en-us/car-manuals/gmc', 'https://haynes.com/en-us/car-manuals/honda', 'https://haynes.com/en-us/car-manuals/hyundai', 'https://haynes.com/en-us/car-manuals/infiniti', 'https://haynes.com/en-us/car-manuals/isuzu', 'https://haynes.com/en-us/car-manuals/jaguar', 'https://haynes.com

In [105]:
#Query the website and return the html to the variable 'page'
list_models = []

for link in list_makes:
    page_temp = urllib2.urlopen(link)
    soup_temp = BeautifulSoup(page_temp)
    model_temp = soup_temp.find_all("div", class_="item-list")[1]
    links_model = model_temp.find_all("li")
    for model in links_model:
        list_models.append(model.a["href"])


In [ ]:
result = []
for link in list_models:
    # 3 types of links
    # 1-result of a search, go to next level
    if "search?query" in link:
        page_temp = urllib2.urlopen(link)
        soup_temp = BeautifulSoup(page_temp)
        model_temp = soup_temp.find_all("div", class_="product-teaser-title")[1]
        for item in model_temp:
            link = prefix + item.a["href"]
            page_1 = urllib2.urlopen(link)
            soup_1 = BeautifulSoup(page_1)
            title = soup_1.find("h1", class_="product-title").get_text()
            price = soup_1.find("span", class_="price-amount commerce-price-savings-formatter-price").get_text()
            body = soup_1.find_all("div", class_="field-item even")
            Format = body[2].get_text()
            decription = body[4].get_text() + body[5].get_text()+ body[6].get_text() + body[7].get_text()
            table = body[8].get_text()
            part_no = body[9].get_text()
            dimensions = body[10].get_text()
            

            
            
    
    #check if this is the final page we are looking for
    model_temp = soup_temp.find_all("div", class_="item-list")[1]
    # 2-need to go to next level by "more info"
    
    # 3-final page
    
    page_temp = urllib2.urlopen(link)
    soup_temp = BeautifulSoup(page_temp)
    
    model_temp = soup_temp.find_all("div", class_="item-list")[1]
    links_model = model_temp.find_all("li")
    for model in links_model:
        list_models.append(model.a["href"])

In [240]:
page_temp = urllib2.urlopen("https://haynes.com/en-us/ford-full-size-econoline-e-100-e-350-petrol-vans-1969-1991-haynes-repair-manual")
soup_temp = BeautifulSoup(page_temp)
#print soup_temp.prettify()
price = soup_temp.find("span", class_="price-amount commerce-price-savings-formatter-price").get_text()
body = soup_temp.find_all("div", class_="field-item even")
Format = body[2].get_text() 
Description = body[9].get_text()

In [241]:
print Description

36090


In [225]:
print body

[<div class="field-item even"><h1 class="product-title">Ford full-size Econoline E-100-E-350 petrol vans (1969-1991) Haynes Repair Manual</h1>\n</div>, <div class="field-item even"><div class="field field-type-commerce-price-element field-type-commerce-price-element-price"><label class="price-label">List Price</label><span class="price-amount commerce-price-savings-formatter-price">$29.95</span></div></div>, <div class="field-item even">Paperback</div>, <div class="field-item even"><form accept-charset="UTF-8" action="/en-us/ford-full-size-econoline-e-100-e-350-petrol-vans-1969-1991-haynes-repair-manual" class="commerce-add-to-cart commerce-cart-add-to-cart-form-632785" id="commerce-cart-add-to-cart-form-632785" method="post"><input name="product_id" type="hidden" value="632785"/>\n<input name="form_build_id" type="hidden" value="form-xdFdTyUI_1KroaHEGjddis8w697Sm-Ov2g2CK41KnH4"/>\n<input name="form_id" type="hidden" value="commerce_cart_add_to_cart_form_632785"/>\n<div class="form-wra